In [ ]:
import h5py
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, butter, filtfilt
import glob
import os
import matplotlib.pyplot as plt
from datetime import datetime

def H(title):
    line = "=" * max(12, len(title))
    print("\n" + line)
    print(title)
    print(line)

H("Imports and runtime state")
print("Libraries loaded")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

H("Global parameters and core assumptions")
RESP_H5_PATH = r"C:\Users\Padilla-Coreano\Documents\tanish\Resp_h5"
EXCEL_PATH = r"C:\Users\Padilla-Coreano\Documents\tanish\Keypoint RI1+RI2\Excel_Results"
PROCESSED_DATA_PATH = r"C:\Users\Padilla-Coreano\Documents\tanis...oint RI1+RI2\Excel_Results\processed_respiration_data_0.5Hz.pkl"

ORIGINAL_SAMPLING_RATE = 20000
TARGET_SAMPLING_RATE = 100
MIN_SYLLABLE_DURATION = 500

PEAK_MIN_DISTANCE = 0.08
PEAK_PROMINENCE_FACTOR = 0.2

FPS_MAPPING = {
    "RI1_s1_1": 15,
    "RI1_s2_3": 15,
    "RI1_s3_6": 29,
    "RI2_s1_1": 15,
    "RI2_s2_4": 15
}
DEFAULT_FPS = 30

print({
    "ORIGINAL_SAMPLING_RATE": ORIGINAL_SAMPLING_RATE,
    "TARGET_SAMPLING_RATE": TARGET_SAMPLING_RATE,
    "MIN_SYLLABLE_DURATION_MS": MIN_SYLLABLE_DURATION,
    "PEAK_MIN_DISTANCE_SEC": PEAK_MIN_DISTANCE,
    "PEAK_PROMINENCE_FACTOR": PEAK_PROMINENCE_FACTOR,
    "DEFAULT_FPS": DEFAULT_FPS,
    "FPS_MAPPING_KEYS": sorted(list(FPS_MAPPING.keys()))
})

H("H5 ingestion primitives")
def list_h5_datasets(h5_path):
    keys = []
    with h5py.File(h5_path, "r") as f:
        def visitor(name, obj):
            if isinstance(obj, h5py.Dataset):
                keys.append(name)
        f.visititems(visitor)
    return sorted(keys)

def read_h5_signal(h5_path, dataset_key):
    with h5py.File(h5_path, "r") as f:
        x = np.array(f[dataset_key])
    return x

print("H5 dataset listing and reading ready")

H("Filtering and resampling primitives")
def butter_lowpass(cutoff_hz, fs_hz, order=4):
    nyq = 0.5 * fs_hz
    normal = cutoff_hz / nyq
    b, a = butter(order, normal, btype="low", analog=False)
    return b, a

def apply_lowpass(x, cutoff_hz, fs_hz, order=4):
    b, a = butter_lowpass(cutoff_hz, fs_hz, order=order)
    return filtfilt(b, a, x)

def resample_by_decimation(x, original_fs, target_fs):
    factor = int(round(original_fs / target_fs))
    if factor <= 0:
        raise ValueError("Invalid decimation factor")
    if original_fs % target_fs != 0:
        raise ValueError("original_fs must be an integer multiple of target_fs for this resampling method")
    return x[::factor]

print("Filter + decimation resampling ready")

H("Normalization primitives (if used before peak detection)")
def zscore(x):
    s = np.std(x)
    m = np.mean(x)
    if s == 0:
        return x * 0
    return (x - m) / s

def center(x):
    return x - np.mean(x)

print("Normalization helpers ready")

H("Peak detection logic")
def detect_breathing_peaks(signal, fs=TARGET_SAMPLING_RATE):
    signal_std = np.std(signal)
    signal_mean = np.mean(signal)

    peaks, properties = find_peaks(
        signal,
        distance=int(fs * PEAK_MIN_DISTANCE),
        prominence=signal_std * PEAK_PROMINENCE_FACTOR,
        height=signal_mean
    )

    return peaks

print("Peak detection ready")

H("Video FPS resolution")
def get_video_fps(filename):
    for pattern, fps in FPS_MAPPING.items():
        if pattern in filename:
            return fps
    return DEFAULT_FPS

print("FPS resolver ready")

H("Respiratory rate and IBI math from peaks")
def calculate_respiratory_metrics(peaks, fs=TARGET_SAMPLING_RATE):
    if len(peaks) < 2:
        return np.array([]), np.array([]), np.array([])

    peak_times = peaks / fs
    ibi_seconds = np.diff(peak_times)
    breathing_rate_1_over_ibi = 1 / ibi_seconds
    breath_times = peak_times[1:]

    return breathing_rate_1_over_ibi, breath_times, ibi_seconds

print("Resp metrics ready")

H("Behavior dataframe schema normalization")
def canonicalize_behavior_columns(df):
    df = df.copy()

    renames = {}
    if "frame_number" not in df.columns:
        if "Frame" in df.columns:
            renames["Frame"] = "frame_number"
        if "frame" in df.columns:
            renames["frame"] = "frame_number"

    if "syllable" not in df.columns:
        if "Syllable" in df.columns:
            renames["Syllable"] = "syllable"
        if "syllable_id" in df.columns:
            renames["syllable_id"] = "syllable"

    if renames:
        df = df.rename(columns=renames)

    if "frame_number" not in df.columns or "syllable" not in df.columns:
        raise ValueError(f"Required columns not found after canonicalization. Columns={list(df.columns)}")

    df = df.dropna(subset=["frame_number", "syllable"])
    df["frame_number"] = df["frame_number"].astype(int)
    df["syllable"] = df["syllable"].astype(int)

    return df

print("Behavior schema normalizer ready")

H("Keypoint-MoSeq syllable bout construction (grouping + duration filtering)")
def create_syllable_bouts(behavior_df, video_fps, min_duration_ms=MIN_SYLLABLE_DURATION):
    behavior_df = behavior_df.copy()
    behavior_df["time_seconds"] = behavior_df["frame_number"] / video_fps
    behavior_df["transition"] = (behavior_df["syllable"] != behavior_df["syllable"].shift()).astype(int)
    behavior_df["bout_group"] = behavior_df["transition"].cumsum()

    bouts_list = []

    for group_id, group in behavior_df.groupby("bout_group"):
        syllable_id = int(group["syllable"].iloc[0])

        start_time = float(group["time_seconds"].min())
        end_time = float(group["time_seconds"].max())
        duration_ms = float((end_time - start_time) * 1000)

        if duration_ms >= min_duration_ms:
            bout_info = {
                "syllable_id": syllable_id,
                "start_frame": int(group["frame_number"].min()),
                "end_frame": int(group["frame_number"].max()),
                "start_time_sec": start_time,
                "end_time_sec": end_time,
                "duration_ms": duration_ms,
                "frame_count": int(len(group)),
                "fps_used": float(video_fps),
            }
            bouts_list.append(bout_info)

    return bouts_list

print("Bout constructor ready")

H("Session key reconciliation and Excel selection")
def find_resp_key(resp_dict, session_pattern):
    matches = [k for k in resp_dict.keys() if session_pattern in k]
    if len(matches) == 0:
        return None
    if len(matches) == 1:
        return matches[0]
    matches_sorted = sorted(matches, key=len)
    return matches_sorted[0]

def pick_excel_file(session_id, excel_files):
    matches = [f for f in excel_files if session_id in os.path.basename(f)]
    if len(matches) == 0:
        return None
    if len(matches) == 1:
        return matches[0]
    matches_sorted = sorted(matches)
    return matches_sorted[0]

print("Key matching helpers ready")

H("Windowing math and per-bout respiratory summaries")
def count_peaks_in_window(peak_times, start_time, end_time, inclusive=True):
    if inclusive:
        mask = (peak_times >= start_time) & (peak_times <= end_time)
    else:
        mask = (peak_times > start_time) & (peak_times < end_time)
    return int(mask.sum())

def summarize_bout_respiration(peak_times, breath_times, breathing_rates_hz, start_time, end_time):
    peak_count = count_peaks_in_window(peak_times, start_time, end_time, inclusive=True)

    mid_time = (start_time + end_time) / 2
    if len(breath_times) > 0 and len(breathing_rates_hz) > 0:
        rate_hz = float(np.interp(mid_time, breath_times, breathing_rates_hz))
        bpm = float(rate_hz * 60)
        ibi = float(1 / rate_hz) if rate_hz > 0 else np.nan
    else:
        rate_hz = np.nan
        bpm = np.nan
        ibi = np.nan

    return {
        "peak_count": peak_count,
        "mean_breathing_rate_hz": rate_hz,
        "mean_bpm": bpm,
        "mean_ibi_sec": ibi
    }

print("Bout respiration summary ready")

H("Single-session integration (peaks + bouts + per-bout features)")
def process_single_session(session_id, respiratory_signal, excel_files):
    print(f"Processing: {session_id}")

    signal = respiratory_signal["respiration"]
    fs = respiratory_signal.get("fs", TARGET_SAMPLING_RATE)

    peaks = detect_breathing_peaks(signal, fs)
    print(f"  Peaks detected: {len(peaks)}")

    if len(peaks) < 2:
        print("  Insufficient peaks - skipping")
        return None

    breathing_rates_hz, breath_times, ibi_values = calculate_respiratory_metrics(peaks, fs)
    peak_times = peaks / fs

    excel_file = pick_excel_file(session_id, excel_files)
    if excel_file is None:
        print(f"  No Excel file found for {session_id}")
        return None

    print(f"  Using Excel file: {os.path.basename(excel_file)}")
    behavior_df = pd.read_excel(excel_file)
    behavior_df = canonicalize_behavior_columns(behavior_df)

    video_fps = get_video_fps(os.path.basename(excel_file))
    bouts_list = create_syllable_bouts(behavior_df, video_fps, MIN_SYLLABLE_DURATION)

    print(f"  Found {len(bouts_list)} syllable bouts")

    processed_bouts = []
    for bout in bouts_list:
        start_time = bout["start_time_sec"]
        end_time = bout["end_time_sec"]
        resp_summary = summarize_bout_respiration(peak_times, breath_times, breathing_rates_hz, start_time, end_time)

        out_row = {
            "syllable_id": bout["syllable_id"],
            "start_time_sec": start_time,
            "end_time_sec": end_time,
            "duration_ms": bout["duration_ms"],
            "frame_count": bout["frame_count"],
            "fps_used": bout["fps_used"],
        }
        out_row.update(resp_summary)
        processed_bouts.append(out_row)

    results_df = pd.DataFrame(processed_bouts)
    results_df = results_df.sort_values("start_time_sec").reset_index(drop=True)

    print(f"  Processed {len(results_df)} syllable bouts")
    return results_df

print("Single-session runner ready")

H("Batch runner and export logic")
def run_complete_analysis():
    print("Starting analysis...")

    respiratory_data = pd.read_pickle(PROCESSED_DATA_PATH)
    excel_files = glob.glob(os.path.join(EXCEL_PATH, "*track0*.xlsx"))

    print(f"Found {len(respiratory_data)} respiratory sessions")
    print(f"Found {len(excel_files)} behavioral files")

    all_results = {}
    successful_sessions = 0

    for session_id, resp_data in respiratory_data.items():
        try:
            result = process_single_session(session_id, resp_data, excel_files)

            if result is not None:
                session_name = session_id.replace(" ", "_").replace("/", "_")
                output_file = f"CORRECTED_PEAK_COUNT_{session_name}_analysis.xlsx"
                result.to_excel(output_file, index=False)
                print(f"  Saved: {output_file}")

                all_results[session_id] = result
                successful_sessions += 1

        except Exception as e:
            print(f"  Error processing {session_id}: {str(e)}")

    print(f"Analysis complete: {successful_sessions}/{len(respiratory_data)} sessions processed")
    return all_results

print("Batch runner ready")

H("Peak-count verification against exported analysis output")
def verify_peak_detection(session_id="RI1_s1_1", test_rows=[2, 5, 8]):
    excel_pattern = f"CORRECTED_PEAK_COUNT_*{session_id}*_analysis.xlsx"
    excel_files = glob.glob(excel_pattern)

    if not excel_files:
        print(f"No Excel file found for {session_id}")
        return None

    excel_file = excel_files[0]
    df = pd.read_excel(excel_file)

    respiratory_data = pd.read_pickle(PROCESSED_DATA_PATH)
    resp_key = find_resp_key(respiratory_data, session_id)
    if resp_key is None:
        print(f"No respiration key found for {session_id}")
        return None

    signal = respiratory_data[resp_key]["respiration"]
    fs = respiratory_data[resp_key].get("fs", TARGET_SAMPLING_RATE)

    peaks = detect_breathing_peaks(signal, fs)
    peak_times = peaks / fs

    matches = 0
    total_tests = len(test_rows)

    plt.figure(figsize=(15, 8))

    for i, row_idx in enumerate(test_rows):
        if row_idx >= len(df):
            continue

        bout = df.iloc[row_idx]
        start_time = float(bout["start_time_sec"])
        end_time = float(bout["end_time_sec"])

        window_mask = (peak_times >= start_time) & (peak_times <= end_time)
        window_peak_times = peak_times[window_mask]

        graph_count = len(window_peak_times)
        excel_count = int(bout["peak_count"])

        if graph_count == excel_count:
            matches += 1
            match_text = "MATCH"
        else:
            match_text = "MISMATCH"

        plt.subplot(len(test_rows), 1, i + 1)

        t = np.arange(len(signal)) / fs
        time_window = (t >= start_time - 0.5) & (t <= end_time + 0.5)
        window_time = t[time_window]
        window_signal = signal[time_window]

        plt.plot(window_time, window_signal)

        for pt in window_peak_times:
            plt.axvline(x=pt, linestyle="--", alpha=0.7)

        plt.axvline(x=start_time, linewidth=2)
        plt.axvline(x=end_time, linewidth=2)

        plt.title(
            f"Row {row_idx}: Excel={excel_count}, Graph={graph_count} {match_text} "
            f"(Syllable {int(bout['syllable_id'])}, {start_time:.2f}-{end_time:.2f}s)"
        )

    plt.suptitle(f"{session_id} Verification - {matches}/{total_tests} matches")
    plt.tight_layout()
    plt.show()

    print(f"Verification: {matches}/{total_tests} syllables match")
    return matches == total_tests

print("Verification routine ready")
